# Agent Automa

In this tutorial, we'll build a **travel planning agent** using the Cognitive Module's three core components: `AgentAutoma`, `CognitiveWorker`, and `CognitiveContext`. You'll learn how to:

- Define mock tools and wrap them as `FunctionToolSpec`
- Create a custom `CognitiveContext` with pre-loaded tools
- Build workers with different thinking modes (`FAST` vs `DEFAULT`)
- Compose multiple agent strategies: React, Plan, Mix, and conditional fallback

## 1. Setup

First, initialize the LLM and import the necessary modules.

In [ ]:
import os

# Set the API base and key.
_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

from bridgic.core.cognitive import (
    CognitiveContext,
    CognitiveWorker,
    ThinkingMode,
    AgentAutoma,
    think_step,
    ErrorStrategy,
)
from bridgic.core.agentic.tool_specs import FunctionToolSpec
from bridgic.llms.openai import OpenAILlm

llm = OpenAILlm(api_base=_api_base, api_key=_api_key, timeout=120)

## 2. Define Tools

Our travel planning agent needs tools to search and book flights and hotels. We define simple mock functions and wrap them using `FunctionToolSpec.from_raw()`, which automatically extracts the tool name, description, and parameter schema from the function signature and docstring.

In [ ]:
def search_flights(origin: str, destination: str) -> str:
    """Search for available flights between two cities.

    Parameters
    ----------
    origin : str
        The departure city.
    destination : str
        The arrival city.
    """
    return f"Found 3 flights from {origin} to {destination}: FL100 ($300), FL200 ($450), FL300 ($280)"


def search_hotels(city: str) -> str:
    """Search for available hotels in a city.

    Parameters
    ----------
    city : str
        The city to search hotels in.
    """
    return f"Found 3 hotels in {city}: Grand Hotel ($120/night), City Inn ($80/night), Budget Stay ($50/night)"


def book_flight(flight_id: str) -> str:
    """Book a specific flight by its ID.

    Parameters
    ----------
    flight_id : str
        The flight identifier (e.g., FL100).
    """
    return f"Successfully booked flight {flight_id}. Confirmation: BK-{flight_id}-2024"


def book_hotel(hotel_name: str, nights: str) -> str:
    """Book a hotel for a specified number of nights.

    Parameters
    ----------
    hotel_name : str
        The name of the hotel to book.
    nights : str
        Number of nights to stay.
    """
    return f"Successfully booked {hotel_name} for {nights} nights. Confirmation: HBK-{hotel_name[:3].upper()}-2024"


# Wrap functions as FunctionToolSpec
travel_tools = [
    FunctionToolSpec.from_raw(search_flights),
    FunctionToolSpec.from_raw(search_hotels),
    FunctionToolSpec.from_raw(book_flight),
    FunctionToolSpec.from_raw(book_hotel),
]

## 3. Custom Context

We extend `CognitiveContext` to pre-load our travel tools. The `__post_init__` hook runs automatically after the context is created, so tools are ready to use immediately.

In [ ]:
class TravelPlanningContext(CognitiveContext):
    """Context with travel planning tools pre-loaded."""

    def __post_init__(self):
        for tool_spec in travel_tools:
            self.tools.add(tool_spec)

## 4. Define Workers

A `CognitiveWorker` represents one "observe-think-act" cycle. The key method to override is `thinking()`, which returns the prompt guiding how the LLM reasons about the next step(s).

We create two workers with different strategies:

- **ReactThinkingWorker** uses `ThinkingMode.FAST` -- thinking and tool selection happen in a single LLM call. It plans **one step at a time**, responding to each result before deciding what to do next.
- **PlanThinkingWorker** uses `ThinkingMode.DEFAULT` -- thinking and tool selection are separate. It creates a **complete plan** covering all steps, then selects tools for each step individually.

In [ ]:
class ReactThinkingWorker(CognitiveWorker):
    """React-style worker: plan ONE step at a time."""

    async def thinking(self) -> str:
        return (
            "You are a reactive assistant. Your approach is to OBSERVE, THINK, then ACT - one step at a time.\n\n"
            "Rather than planning everything upfront, you respond to the current situation and take "
            "the single most appropriate next action. After each action, you observe the result and "
            "decide what to do next based on the new information.\n\n"
            "Your reactive process:\n"
            "1. Observe the current state\n"
            "2. Determine the single best next action\n"
            "3. Execute that one action and wait for the result\n"
            "4. Repeat until the goal is achieved\n\n"
            "Focus only on the immediate next step. Do not plan multiple steps ahead."
        )


class PlanThinkingWorker(CognitiveWorker):
    """Plan-style worker: create a COMPLETE plan."""

    async def thinking(self) -> str:
        return (
            "You are a planning-oriented assistant. Your approach is to THINK BEFORE YOU ACT.\n\n"
            "Before taking any action, create a complete plan covering all steps needed to achieve the goal. "
            "Do not just consider the immediate next step - think through the ENTIRE task from beginning to end.\n\n"
            "Your planning process:\n"
            "1. Understand what the user wants to accomplish\n"
            "2. Assess current progress\n"
            "3. Identify all remaining work needed\n"
            "4. Break down into concrete, actionable steps\n\n"
            "Important: If some work has already been completed, build upon that progress. "
            "Only plan the steps that still need to be done."
        )

## 5. ReactAgent

Now let's build our first agent. An `AgentAutoma` subclass defines:

1. **Thinking steps** -- class-level attributes created with `think_step()`, each wrapping a `CognitiveWorker`
2. **`cognition()` method** -- the orchestration logic using plain Python async control flow

The `ReactAgent` loops the react step up to 5 times, checking `ctx.finish` after each iteration. It also compresses history when it grows too large.

In [ ]:
class ReactAgent(AgentAutoma[TravelPlanningContext]):
    """React-style agent that loops until the goal is achieved."""

    react = think_step(
        ReactThinkingWorker(llm=llm, mode=ThinkingMode.FAST, verbose=True),
        on_error=ErrorStrategy.RAISE,
    )

    async def cognition(self, ctx: TravelPlanningContext):
        for _ in range(5):
            await self.react
            await ctx.cognitive_history.compress_if_needed()
            if ctx.finish:
                break

In [ ]:
agent = ReactAgent(llm=llm)
result = await agent.arun(
    goal="I'm currently in Beijing and I want to go to Kunming tomorrow. Could you help me plan the route and hotel?"
)

print(f'\n- - - - - Result - - - - -')
print(result)

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 6. PlanAgent

The `PlanAgent` uses `ThinkingMode.DEFAULT`, which separates thinking (what steps to take) from tool selection (which tools to use for each step). In a single `cognition()` call, the worker:

1. **Thinks**: produces a multi-step plan (e.g., search flights, book flight, search hotels, book hotel)
2. **Acts**: for each planned step, selects appropriate tools via a separate LLM call, then executes them

This two-phase approach works well when you want the agent to plan comprehensively before acting.

In [ ]:
class PlanAgent(AgentAutoma[TravelPlanningContext]):
    """Plan-style agent that executes a single planning step."""

    plan = think_step(
        PlanThinkingWorker(llm=llm, mode=ThinkingMode.DEFAULT, verbose=True)
    )

    async def cognition(self, ctx: TravelPlanningContext):
        await self.plan

In [ ]:
agent = PlanAgent(llm=llm)
result = await agent.arun(
    goal="I'm currently in Beijing and I want to go to Kunming. Could you help me plan the route?"
)

print(f'\n- - - - - Result - - - - -')
print(result)

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 7. MixAgent

Since `cognition()` is just plain Python, you can freely mix different strategies. The `MixAgent` runs two React steps first (quick reactive actions), then switches to a Plan step for comprehensive completion.

In [ ]:
class MixAgent(AgentAutoma[TravelPlanningContext]):
    """Mixed agent: two React steps, then one Plan step."""

    react = think_step(
        ReactThinkingWorker(llm=llm, mode=ThinkingMode.FAST, verbose=True),
    )
    plan = think_step(
        PlanThinkingWorker(llm=llm, mode=ThinkingMode.DEFAULT, verbose=True)
    )

    async def cognition(self, ctx: TravelPlanningContext):
        await self.react
        await self.react
        await self.plan

In [ ]:
agent = MixAgent(llm=llm)
result = await agent.arun(
    goal="I'm currently in Beijing and I want to go to Kunming. Could you help me plan the route?"
)

print(f'\n- - - - - Result - - - - -')
print(result)

<div style="text-align: center; margin: 2rem 0;"><hr style="border: none; border-top: 2px solid #e2e8f0;"></div>

## 8. ReactThenPlanAgent

A more sophisticated pattern: try the React approach first (up to 3 iterations). If the task isn't finished and the agent has accumulated some history, fall back to Plan mode for a comprehensive completion. This demonstrates **conditional branching** in `cognition()`.

In [ ]:
class ReactThenPlanAgent(AgentAutoma[TravelPlanningContext]):
    """React until stuck, then Plan to finish."""

    react = think_step(
        ReactThinkingWorker(llm=llm, mode=ThinkingMode.FAST, verbose=True)
    )
    plan = think_step(
        PlanThinkingWorker(llm=llm, mode=ThinkingMode.DEFAULT, verbose=True)
    )

    async def cognition(self, ctx: TravelPlanningContext):
        # First try React approach (up to 3 times)
        for _ in range(3):
            await self.react
            if ctx.finish or len(ctx.cognitive_history) >= 3:
                break

        # If not finished, use Plan to complete the rest
        if not ctx.finish:
            await self.plan

In [ ]:
agent = ReactThenPlanAgent(llm=llm)
result = await agent.arun(
    goal="I'm currently in Beijing and I want to go to Kunming tomorrow. Could you help me plan the route and hotel?"
)

print(f'\n- - - - - Result - - - - -')
print(result)

## What have we learnt?

| Concept | Description |
|---------|-------------|
| `CognitiveWorker` | One "observe-think-act" cycle. Override `thinking()` to control reasoning. |
| `ThinkingMode.FAST` | Merges thinking + tool selection into one LLM call. Best for React-style, single-step agents. |
| `ThinkingMode.DEFAULT` | Separates thinking and tool selection into two phases. Best for Plan-style, multi-step agents. |
| `think_step()` | Wraps a `CognitiveWorker` as an awaitable step on `AgentAutoma`. Supports `on_error`, `tools`, and `skills` filtering. |
| `AgentAutoma` | Orchestrates workers via `cognition()`. Uses plain Python async (`for`, `while`, `if/else`) for full control. |
| `CognitiveContext` | Holds goal, tools, skills, and history. Extend with `__post_init__` to pre-load data. |
| `FunctionToolSpec.from_raw()` | Wraps a Python function as a tool, auto-extracting name, description, and parameter schema. |
| `ErrorStrategy` | Controls error handling per step: `RAISE`, `IGNORE`, or `RETRY`. |
| `ctx.finish` | Set to `True` when the LLM decides the goal is achieved. Use it as a loop termination condition. |